In [ ]:
import pandas as pd
import numpy as np
from numpy import savez_compressed
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
import os
import joblib

In [ ]:
!gdown https://drive.google.com/uc?id=1oCIVZbIWmg2RvcCHcxko5alyZh19nGoF
!gdown https://drive.google.com/uc?id=1vCS6gDbsS3mVE5WLHM0nu-ww0GcI1ybN
!gdown https://drive.google.com/uc?id=10_pO2fGiAKSdNjO-p3d1dFo3VuY1juL5
!gdown https://drive.google.com/uc?id=1kSH1dteCOdFT5eFFM-1DCdbl7TZ4gXzS
!gdown https://drive.google.com/uc?id=1nRKTC6Pez1fRAMYkFIHQNLfB0Hhj5fok
!gdown https://drive.google.com/uc?id=1lA19-bvYJ_btc1WSXpKqU9L6nD3dpUjR
!gdown https://drive.google.com/uc?id=1Av_X9N5BB38edPLK0vtYIBQ9toUdDB1e
!gdown https://drive.google.com/uc?id=11u-MTRj8qiqC0rsB23F37lc4MjNh3Fhk

Downloading...
From: https://drive.google.com/uc?id=1oCIVZbIWmg2RvcCHcxko5alyZh19nGoF
To: /content/Monday-WorkingHours.pcap_ISCX.csv
177MB [00:01, 105MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1vCS6gDbsS3mVE5WLHM0nu-ww0GcI1ybN
To: /content/Tuesday-WorkingHours.pcap_ISCX.csv
135MB [00:01, 104MB/s] 
Downloading...
From: https://drive.google.com/uc?id=10_pO2fGiAKSdNjO-p3d1dFo3VuY1juL5
To: /content/Wednesday-workingHours.pcap_ISCX.csv
225MB [00:01, 127MB/s]
Downloading...
From: https://drive.google.com/uc?id=1kSH1dteCOdFT5eFFM-1DCdbl7TZ4gXzS
To: /content/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
52.0MB [00:00, 72.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1nRKTC6Pez1fRAMYkFIHQNLfB0Hhj5fok
To: /content/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
83.1MB [00:00, 116MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1lA19-bvYJ_btc1WSXpKqU9L6nD3dpUjR
To: /content/Friday-WorkingHours-Morning.pcap_ISCX.csv
58.3MB [00:00, 103MB/s

In [ ]:
datasets = os.listdir()
datasets.remove('.config')
datasets.remove('sample_data')
datasets

['Friday-WorkingHours-Morning.pcap_ISCX.csv',
 'Monday-WorkingHours.pcap_ISCX.csv',
 'Wednesday-workingHours.pcap_ISCX.csv',
 'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv',
 'Tuesday-WorkingHours.pcap_ISCX.csv',
 'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv',
 'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv',
 'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv']

In [ ]:
df = pd.read_csv(datasets[0])

for i in range(1, len(datasets)):
  df_inter = pd.read_csv(datasets[i])
  df = pd.concat([df, df_inter], axis=0)
  
df = df.sample(frac = 1)

In [ ]:
imp_features = ['Flow IAT Min',
 'Bwd IAT Mean',
 'Flow IAT Mean',
 'Init_Win_bytes_forward',
 'Subflow Fwd Bytes',
 'Total Length of Fwd Packets',
 'ACK Flag Count',
 'Fwd PSH Flags',
 'SYN Flag Count',
 'Fwd Packets/s',
 'Init_Win_bytes_backward',
 'Flow Duration',
 'Active Mean',
 'Fwd Packet Length Mean',
 'Bwd Packets/s',
 'PSH Flag Count',
 'Bwd Packet Length Std',
 'Average Packet Size',
 'Flow IAT Std',
 'Label']

In [ ]:
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
333356,55033,45,1,1,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.000000e+00,44444.444440,45.0,0.000000,45,45,0,0.0,0.000000,0,0,0,0.0,0.0,0,0,0,0,0,0,32,32,22222.222220,22222.222220,0,0,0.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,1,0.0,0.0,0.0,32,0,0,0,0,0,0,1,0,1,0,122,33304,0,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
230743,1022,61,1,1,0,6,0,0,0.0,0.0,6,6,6.0,0.0,9.836066e+04,32786.885250,61.0,0.000000,61,61,0,0.0,0.000000,0,0,0,0.0,0.0,0,0,0,0,0,0,40,20,16393.442620,16393.442620,0,6,2.000000,3.464102,12.000000,0,0,0,1,0,0,0,0,1,3.0,0.0,6.0,40,0,0,0,0,0,0,1,0,1,6,29200,0,0,40,0.0,0.0,0,0,0.0,0.0,0,0,PortScan
29862,443,10254,3,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.000000e+00,292.568754,5127.0,7249.258721,10253,1,10254,5127.0,7249.258721,10253,1,0,0.0,0.0,0,0,0,0,0,0,96,0,292.568754,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,96,0,0,0,0,0,0,3,0,0,0,322,-1,0,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
480184,53,198,2,2,80,204,40,40,40.0,0.0,102,102,102.0,0.0,1.434343e+06,20202.020200,66.0,30.315013,101,48,49,49.0,0.000000,49,49,48,48.0,0.0,48,48,0,0,0,0,64,52,10101.010100,10101.010100,40,102,64.800000,33.958799,1153.200000,0,0,0,0,0,0,0,0,1,81.0,40.0,102.0,64,0,0,0,0,0,0,2,80,2,204,-1,-1,1,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
172867,53,33919,1,1,43,71,43,43,43.0,0.0,71,71,71.0,0.0,3.360948e+03,58.964002,33919.0,0.000000,33919,33919,0,0.0,0.000000,0,0,0,0.0,0.0,0,0,0,0,0,0,40,32,29.482001,29.482001,43,71,52.333333,16.165808,261.333333,0,0,0,0,0,0,0,0,1,78.5,43.0,71.0,40,0,0,0,0,0,0,1,43,1,71,-1,-1,0,40,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [ ]:
cols = list(df.columns)

for col in cols:
  if col[0] == ' ':
    df = df.rename(columns={col: col[1:]})

In [ ]:
#df = df[imp_features]
cols = list(df.columns)

In [ ]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
df = df.reset_index(drop=True)

In [ ]:
df.iloc[:,-1] = df.iloc[:, -1].apply(lambda x : 'Web' if x[:3] == 'Web' or x[:3] == 'Sql' else x)

In [ ]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
ss = StandardScaler()
X = ss.fit_transform(X)

In [ ]:
X

array([[ 2.57035422, -0.43961119, -0.01115785, ..., -0.10951643,
        -0.35707153, -0.33918443],
       [-0.3852146 , -0.43961071, -0.01115785, ..., -0.10951643,
        -0.35707153, -0.33918443],
       [-0.41689841, -0.43930796, -0.00849137, ..., -0.10951643,
        -0.35707153, -0.33918443],
       ...,
       [-0.43823984, -0.43888741, -0.00982461, ..., -0.10951643,
        -0.35707153, -0.33918443],
       [-0.43823984, -0.43960757, -0.00982461, ..., -0.10951643,
        -0.35707153, -0.33918443],
       [-0.43823984, -0.43765414, -0.00982461, ..., -0.10951643,
        -0.35707153, -0.33918443]])

In [ ]:
np.unique(y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 21)

In [ ]:
#savez_compressed('y_test.npz', y_test)

In [ ]:
def get_integer_mapping(le):
    '''
    Return a dict mapping labels to their integer values
    from an SKlearn LabelEncoder
    le = a fitted SKlearn LabelEncoder
    '''
    res = {}
    for cl in le.classes_:
        res.update({cl:le.transform([cl])[0]})

    return res

integerMapping = get_integer_mapping(le)

In [ ]:
rfClassifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)
rfClassifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
y_pred_rf = rfClassifier.predict(X_test)
accuracy_score(y_test, y_pred_rf)

0.9990183445101554

In [ ]:
#savez_compressed('y_pred_rf.npz', y_pred_rf)

In [ ]:
np.unique(y_pred_rf, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
 array([567461,    369,  32118,   2616,  57666,   1342,   1443,   1926,
             4,      4,  39998,   1495,    527]))

In [ ]:
nbClassifier = GaussianNB()
nbClassifier.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
y_pred_nb = nbClassifier.predict(X_test)
accuracy_score(y_test, y_pred_nb)

0.6925310727910277

In [ ]:
#savez_compressed('y_pred_nb.npz', y_pred_nb)

In [ ]:
model_nn = Sequential()

model_nn.add(Dense(32, activation = 'relu', input_dim = 78))
model_nn.add(Dense(64, activation = 'relu'))

model_nn.add(Dense(13, activation = 'softmax'))

In [ ]:
Adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)

In [ ]:
model_nn.compile(loss='categorical_crossentropy', optimizer=Adam, metrics=['accuracy'])

In [ ]:
y_train_nn = to_categorical(y_train)

In [ ]:
model_nn.fit(X_train, y_train_nn, epochs = 2, validation_split = 0.2)

Epoch 1/2
53023/53023 [==============================] - 90s 2ms/step - loss: 0.1094 - accuracy: 0.9635 - val_loss: 0.0783 - val_accuracy: 0.9737
Epoch 2/2
53023/53023 [==============================] - 90s 2ms/step - loss: 0.0944 - accuracy: 0.9715 - val_loss: 0.1048 - val_accuracy: 0.9659


In [ ]:
y_test_nn = to_categorical(y_test)
model_nn.evaluate(X_test, y_test_nn)

22093/22093 [==============================] - 24s 1ms/step - loss: 0.1090 - accuracy: 0.9664


[0.10899565368890762, 0.9664285182952881]

In [ ]:
y_pred_nn = model_nn.predict(X_test)

In [ ]:
y_pred_nn[0]

array([1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       2.5259125e-36, 3.1675005e-15, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00], dtype=float32)

In [ ]:
y_pred_nn.shape

(706969, 13)

In [ ]:
y_pred_nn = np.argmax(y_pred_nn, axis=1)

In [ ]:
#savez_compressed('y_pred_nn.npz', y_pred_nn)

In [ ]:
#model_nn.save("model_nn.h5")
#joblib.dump(rfClassifier, "RF_compressed.joblib", compress=3) 
#joblib.dump(nbClassifier, "NB_compressed.joblib", compress=3) 